In [63]:
import chardet
import numpy as np
import scipy as sp
import scipy.constants as constants
import pandas as pd

R = 0.20
mu_0 = constants.mu_0

e = constants.e
m_e = constants.m_e

valorTeorico = e/m_e

print(valorTeorico)
print(mu_0)

n = 154.0

raios = [2, 3, 4, 5]

amp = []
ampPerRadius = []

volt = []
voltPerRadius = []

magneticField = []    
magneticFieldPerRadius = []

for raio in raios:
    with open(f'data/raio{raio}cm.txt', 'rb') as f:
        enc = chardet.detect(f.read())  # or readline if the file is large

    file = pd.read_csv(f'data/raio{raio}cm.txt', header = None, sep='\s+', encoding=enc['encoding'], dtype = np.float64)
    
    for data in [file[0]]:
        amp.append(data)
        B = np.sqrt((4.0/5.0)*(4.0/5.0)*(4.0/5.0)) * ((mu_0 * n * (data))/R)
        magneticField.append(B)
        
    for data in [file[1]]:
        volt.append(data)
        
    magneticFieldPerRadius.append(magneticField)
    ampPerRadius.append(amp)
    voltPerRadius.append(volt)
    
    amp = []
    volt = []
    magneticField = []

#print(ampPerRadius)
#print(voltPerRadius)
#print(magneticFieldPerRadius)

175882001077.2163
1.25663706212e-06


In [64]:
raios = [0.02, 0.03, 0.04, 0.05]

chargePerMassPerRadius = []
chargePerMass = []

for i in range(np.size(voltPerRadius)):
    j = 0
    for volt in voltPerRadius[i]:
        CpM = 2.0*(volt)/(raios[i]*raios[i]*(magneticFieldPerRadius[i][j])*(magneticFieldPerRadius[i][j]))
        chargePerMass.append(CpM)
        j = j + 1
        
    chargePerMassPerRadius.append(chargePerMass)
    chargePerMass = []
    
#print(chargePerMassPerRadius)

c:\Python310\lib\site-packages\numpy\core\fromnumeric.py:3245: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  return asarray(a).size


In [69]:
tab = '\t'

raio = []
volts = []
amps = []
mF = []
CpM = []

for i in range(np.size(chargePerMassPerRadius)):
       
       for volt in voltPerRadius[i]:
              for k in range(np.size(volt)):
                     volts.append(volt[k])
                     
       for amp in ampPerRadius[i]:
              for k in range(np.size(amp)):
                     amps.append(amp[k])
              
       for campo in magneticFieldPerRadius[i]:
              for k in range(np.size(campo)):
                     mF.append(campo[k])
              
       for razao in chargePerMassPerRadius[i]:
              for k in range(np.size(razao)):
                     CpM.append(razao[k])
              
       dados = {"voltagens": volts,
       "amperagens": amps,
       "campo": mF,
       "razaoE/M": CpM}
       
       dataFrame = pd.DataFrame(dados)
       
       j = i + 2

       dataFrame.to_csv(f'results/dadosRaio{j}cm.dat', index=False, sep=tab)
       
       volts = []
       amps = []
       mF = []
       CpM = []   

c:\Python310\lib\site-packages\numpy\core\fromnumeric.py:3245: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  return asarray(a).size


In [74]:
raios = [2, 3, 4, 5]

for raio in raios:
    file = pd.read_csv(f'results/dadosRaio{raio}cm.dat', header = 0, sep='\s+', dtype = np.float64)
    
    media = file['razaoE/M'].mean()
    desvioPadrao = file['razaoE/M'].std()
    
    erroPercentual = (abs(media - valorTeorico)/valorTeorico)*100
    
    resultados = {"mediaE/M": [media],
    "desvioPadraoE/M": [desvioPadrao],
    "erroPercentual": [erroPercentual]}
    
    dataFrame = pd.DataFrame(resultados)
    
    dataFrame.to_csv(f'results/resultadosRaio{raio}cm.dat', index=False, sep=tab)

162855709311.20187
6110430934.005543
7.406267660268198
162406941899.66785
17804685340.88254
7.661420210719911
171255795604.17935
2252848257.9449735
2.6302893102779428
175081674523.47476
8375960443.743953
0.4550360746635948
